# Programming for Data Science - Final Project

**Dataset :** Volcanic Eruptions (NCEI)

**Team leader :** BENADDI, Youssef

**Team members :**
* BENDAVID, Maxime
* ABOUHACHIM ALAMI, Ali
* BERNHARD, Tom
* BEZIOU, Gabriel

---
**Project objective :**
Analyze volcanic activity patterns and data quality over time and create a dashboard visualization of spatial and temporal eruption indicators, as well as data accuracy indicators.

### 0. PIP INSTALLS

In [ ]:
!pip install dash
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 48.4 MB/s eta 0:00:00


# 1. IMPORTS

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import warnings
import os
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# 2. GLOBAL CONSTANTS

In [ ]:
DATASET_PATH = 'volcano-events-2024-12-17_11-00-15_+0100.tsv'
TEAM_MEMBERS = "BENDAVID, Maxime; ABOUHACHIM ALAMI, Ali; BERNHARD, Tom; BEZIOU, Gabriel"
TEAM_LEADER = "BENADDI, Youssef"
PROJECT_SUBJECT = "Analyzing the evolution of volcanic explosiveness over time: Is activity increasing, or is this an effect of observation bias?"

# VEI Labels (0-8)
VEI_LABELS = {
    -1: 'Unknown/Unrecorded', 0: '0: Effusive', 1: '1: Gentle', 2: '2: Explosive',
    3: '3: Catastrophic', 4: '4: Cataclysmic', 5: '5: Paroxysmic',
    6: '6: Colossal', 7: '7: Super-colossal', 8: '8: Mega-colossal'
}

# Sort labels for the legend
SORTED_VEI_LABELS = [VEI_LABELS[i] for i in sorted(VEI_LABELS.keys()) if i != -1]

# 3. KDD PROCESS FUNCTIONS

In [ ]:
def load_data(file_path):
    """Reads the TSV file."""
    if not os.path.exists(file_path):
        print(f"Error: File not found at {file_path}")
        return None
    try:
        return pd.read_csv(file_path, sep='\t', header=0, encoding='utf-8')
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

In [ ]:
def data_collection_exploration(df):
    """
    Data collection: Explores the data using summarization methods.
    Shows columns, data types, shape, missing values, and ranking (stats).
    """
    # Number of rows and columns
    rows, cols = df.shape
    print(f"\n[INFO] Dataset dimensions: {rows} rows, {cols} columns")

    # Columns and data types
    print("\n[INFO] Data types per column:")
    print(df.dtypes)

    # Missing data points
    print("\n[INFO] Missing values per column:")
    missing = df.isnull().sum()
    print(missing[missing > 0])  # Show only columns with missing values

    # Ranking between variables (statistical summary)
    print("\n[INFO] Statistical summary (ranking):")
    print(df.describe().transpose())

## Cleaning & Transformation Pipeline

In [ ]:
def initial_cleaning(df):
    """Drops rows missing critical Year/Lat/Lon data."""
    df = df.copy()
    # Drop rows where 'Year', 'Latitude', or 'Longitude' are missing as they are fundamental for analysis
    return df.dropna(subset=['Year', 'Latitude', 'Longitude'])

def feature_reduction(df):
    """Drops text columns but keeps impact/metric columns."""
    df = df.copy()
    # Define keywords to identify columns to be dropped (e.g. : descriptive text, internal IDs)
    keywords = ['description', 'search_parameters', '[]', 'agent', 'missing', 'injuries', 'mo', 'dy']
    # Create a list of column names to drop if they contain any of the keywords
    cols_to_drop = [c for c in df.columns if any(k in c for k in keywords)]
    # Drop the identified columns, ignoring errors if a column doesn't exist
    return df.drop(columns=cols_to_drop, errors='ignore')

def impute_numerical(df):
    """Imputes missing VEI/Elevation with Median."""
    df = df.copy()
    # Columns targeted for median imputation
    cols = ['elevation_(m)', 'vei']
    imputer = SimpleImputer(strategy='median')
    for col in cols:
        if col in df.columns:
            # Convert column to numeric, coercing errors to NaN for imputation
            df[col] = pd.to_numeric(df[col], errors='coerce')
            # Apply median imputation and flatten the resulting array
            df[col] = imputer.fit_transform(df[[col]]).ravel()
    return df

def fill_flags(df):
    """Fills missing Tsunami/Eq flags with 0."""
    df = df.copy()
    # Columns representing binary flags (tsunami, earthquake)
    for col in ['tsu', 'eq']:
        if col in df.columns:
            # Fill any missing values with 0, assuming absence means no flag
            df[col] = df[col].fillna(0)
    return df

def normalize_spatial(df):
    """Normalizes spatial coordinates (0-1)."""
    df = df.copy()
    scaler = MinMaxScaler()
    # Columns representing spatial coordinates and elevation for normalization
    for col in ['latitude', 'longitude', 'elevation_(m)']:
        if col in df.columns:
            # Apply Min-Max scaling to normalize values between 0 and 1
            df[f'{col}_norm'] = scaler.fit_transform(df[[col]])
    return df

def calculate_data_completeness(df):
    """
    Creates metrics for observation bias analysis.
    Runs before columns are dropped.
    """
    df = df.copy()
    # Standardize column names to lowercase, strip whitespace, and replace spaces with underscores
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')

    # Ensure 'year' is numeric and calculate 'century'
    df['year'] = pd.to_numeric(df['year'], errors='coerce')
    df['century'] = (df['year'] // 100) * 100

    # Create binary flags for data completeness (1 if data exists, 0 if missing)
    df['has_vei'] = df['vei'].notna().astype(int)
    df['has_deaths'] = df['total_deaths'].notna().astype(int)
    df['has_damage'] = df['total_damage_($mil)'].notna().astype(int)

    # Map the VEI numerical values to descriptive labels for better visualization
    df['vei_label'] = df['vei'].fillna(-1).round().map(VEI_LABELS)

    return df

def add_spatial_clusters(df, n_clusters=5):
    """
    Applies K-Means clustering to spatial coordinates.
    Adds a 'spatial_cluster' column to the dataframe.
    """
    df = df.copy()
    # Select coordinates
    X = df[['latitude', 'longitude']]

    # Apply K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['spatial_cluster'] = kmeans.fit_predict(X)

    # Convert to string for categorical coloring in plots
    df['spatial_cluster'] = df['spatial_cluster'].astype(str)

    return df

def get_indicator_spatial_clusters(df):
    """
    Visualizes the spatial clusters on a map.
    """
    fig = px.scatter_geo(
        df,
        lat='latitude',
        lon='longitude',
        color='spatial_cluster',
        hover_name='name',
        title='Spatial Clustering of Volcanoes (K-Means)',
        projection='natural earth',
        color_discrete_sequence=px.colors.qualitative.Prism
    )
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    return fig

In [ ]:
def build_pipeline():
    """Compiles the cleaning steps."""
    return Pipeline([
        ('initial_clean', FunctionTransformer(initial_cleaning)),
        ('data_metrics', FunctionTransformer(calculate_data_completeness)),
        ('feature_reduction', FunctionTransformer(feature_reduction)),
        ('imputation', FunctionTransformer(impute_numerical)),
        ('fill_flags', FunctionTransformer(fill_flags)),
        ('normalization', FunctionTransformer(normalize_spatial)),
        ('clustering', FunctionTransformer(add_spatial_clusters))
    ])

Based on the data exploration results, the dataset consists of 888 rows and 35 columns, featuring a mix of numerical (float64) and categorical (object) variables.  

A key observation is the stark contrast in data completeness: the core spatiotemporal attributes (Year, Latitude, Longitude, and Elevation) are nearly complete with only a single missing value each, whereas impact-related columns (such as Deaths, Injuries, and Damage) are highly sparse, with missing values ranging from roughly 400 to over 800 entries.  

Statistically, the recorded eruptions span a vast timeline from 4360 BC to 2024 AD, occurring at an average elevation of 1,954 meters, and typically registering a Volcanic Explosivity Index (VEI) of roughly 2.9, with a maximum recorded intensity of 7.  

This confirms that while the geological data is robust for analyzing spatiotemporal cycles, the socio-economic data is too sparse for reliable analysis, validating the decision to remove those columns in the cleaning phase.

# 4. INDICATOR VISUALIZATION FUNCTIONS

In [ ]:
df_raw = load_data(DATASET_PATH)
if df_raw is not None:
    # Perform initial data exploration and display summary statistics
    data_collection_exploration(df_raw)

    # Build and apply the data cleaning and transformation pipeline
    pipeline = build_pipeline()
    df_final = pipeline.fit_transform(df_raw)


[INFO] Dataset dimensions: 888 rows, 35 columns

[INFO] Data types per column:
Search Parameters                      object
Year                                  float64
Mo                                    float64
Dy                                    float64
Tsu                                   float64
Eq                                    float64
Name                                   object
Location                               object
Country                                object
Latitude                              float64
Longitude                             float64
Elevation (m)                         float64
Type                                   object
VEI                                   float64
Agent                                  object
Deaths                                float64
Death Description                     float64
Missing                               float64
Missing Description                   float64
Injuries                              float64


## I. Frequent Pattern Mining (apriori)

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules

def get_indicator_pattern_mining(df):
    """
    Generates a Sankey diagram showing the strongest Association Rules.
    Visualizes the flow from Antecedent (Left) -> Consequent (Right).
    Link thickness represents the Confidence of the rule.
    """
    # Prepare Data: Select categorical columns for mining
    # Combining Type, VEI Label, and Region/Country for interesting patterns
    features = ['type', 'vei_label', 'region'] # Added region for more context

    # Check if columns exist, otherwise fallback
    available_features = [c for c in features if c in df.columns]
    df_mining = df[available_features].copy().dropna()

    # One-Hot Encoding
    df_ohe = pd.get_dummies(df_mining)

    # Apply Apriori Algorithm
    # Lower support slightly to ensure we get enough connections for the diagram
    frequent_itemsets = apriori(df_ohe, min_support=0.02, use_colnames=True)

    if frequent_itemsets.empty:
        return go.Figure().update_layout(title="No patterns found (Try lowering support)")

    # Generate Rules
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

    # Filter: Get top 15 rules by confidence to keep the diagram readable
    top_rules = rules.sort_values('confidence', ascending=False).head(15)

    # Prepare Nodes (Unique items involved in the rules)
    # We need to extract string names from the frozensets
    top_rules['antecedent_str'] = top_rules['antecedents'].apply(lambda x: list(x)[0])
    top_rules['consequent_str'] = top_rules['consequents'].apply(lambda x: list(x)[0])

    # Create a list of all unique nodes (both left and right sides)
    all_nodes = list(set(top_rules['antecedent_str'].unique()) | set(top_rules['consequent_str'].unique()))

    # Map node names to indices (0 to N)
    node_map = {name: i for i, name in enumerate(all_nodes)}

    # Prepare Links (Source -> Target)
    source_indices = top_rules['antecedent_str'].map(node_map).tolist()
    target_indices = top_rules['consequent_str'].map(node_map).tolist()
    conf_values = top_rules['confidence'].tolist()

    # Create Sankey Figure
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=15,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=all_nodes,
            color="teal"
        ),
        link=dict(
            source=source_indices,
            target=target_indices,
            value=conf_values,
            # Link color can be static or based on value, using simple gray here for clarity
            color='rgba(128, 128, 128, 0.4)'
        )
    )])

    fig.update_layout(
        title_text="Association Rules Flow (Confidence > 50%)",
        font_size=12
    )
    return fig

In [ ]:
get_indicator_pattern_mining(df_final)

### **Indicator 1: Association Rules Flow.**
This Sankey diagram visualizes the probability flow of the Apriori algorithm.  
It maps the flow from specific VEI intensities (Antecedents on the left) to the Volcano Type (Consequent on the right)   
Width of the link = Confidence of the rule.  

The diagram reveals a massive convergence: VEI 2 (Explosive), VEI 3 (Catastrophic), VEI 4 (Cataclysmic), and even VEI 5 (Paroxysmic) all flow strongly into Stratovolcanoes.  

This visually proves that Stratovolcanoes are the primary geological drivers of explosive volcanic activity recorded in history. The width of the bands suggests a high confidence level for these rules, confirming that if a high-VEI event occurs, it is almost certainly a Stratovolcano

## II. Grouping (Global ranking)

In [ ]:
def get_indicator_grouping(df):
    """
    Generates a bar chart showing the global ranking of countries by the number of volcanic eruptions.

    This indicator provides a geographical context by highlighting countries with the most volcanic activity,
    which is crucial for understanding the distribution of volcanic events globally.

    Args:
        df (pd.DataFrame): The preprocessed dataframe containing volcanic eruption data.

    Returns:
        plotly.graph_objects.Figure: A plotly bar chart displaying the top countries by eruption count.
    """
    # Calculate the number of eruptions per country
    activity = df['country'].value_counts().reset_index()
    activity.columns = ['Country', 'Eruptions'] # Rename columns for clarity in the plot

    # Create a bar chart using plotly express
    fig = px.bar(
        activity, # Use the computed activity DataFrame
        x='Country', y='Eruptions',
        title='Global ranking of volcanic activity (zoom/pan to explore)',
        color='Eruptions', # Color bars based on the number of eruptions
        color_continuous_scale='Viridis' # Use a viridis color scale
    )
    # Set initial view to display the top 10 countries, allowing users to pan and zoom for more data
    fig.update_xaxes(range=[-0.5, 9.5])
    return fig

In [ ]:
get_indicator_grouping(df_final)

### **Indicator 2: Grouping - Global Activity Ranking**
This chart identifies the countries with the highest number of volcanic events.  
It establishes the major areas of focus for volcanic activity globally and provides the necessary **geographical context** for understanding where volcanic activity is concentrated, helping to interpret the spatial map (Indicator 2).

## III. Spatial (Map)

In [ ]:
def get_indicator_spatial(df):
    """
    Generates a scatter geo map showing the global seismic potential based on the
    maximum recorded Volcanic Explosivity Index (VEI) for each volcano.

    This indicator visualizes the spatial distribution of volcanic activity and
    the intensity of past eruptions, helping to identify high-risk areas globally.
    The size and color of markers are proportional to the maximum VEI.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing volcanic eruption data.
            It should include 'name', 'country', 'vei', 'latitude', and 'longitude' columns.

    Returns:
        plotly.graph_objects.Figure: A Plotly scatter geo map displaying volcanoes
        with their maximum recorded VEI.
    """
    # Max VEI per volcano
    volcano_stats = df.groupby(['name', 'country']).agg({
        'vei': 'max', 'latitude': 'first', 'longitude': 'first'
    }).reset_index()

    fig = px.scatter_geo(
        volcano_stats, lat='latitude', lon='longitude', size='vei', color='vei',
        hover_name='name', title='Global Seismic Potential (Max Recorded VEI)',
        projection='natural earth', color_continuous_scale='Redor'
    )
    # Update margin to make map larger
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    return fig

In [ ]:
get_indicator_spatial(df_final)

### **Indicator 3.A: Spatial - Global Explosive Potential**
The map plots every recorded volcano, with the marker size and color representing the maximum recorded **Volcanic Explosivity Index (VEI)**.  


The largest (highest VEI) eruptions, indicated by the largest and darkest red markers (VEI 6 and 7), are highly concentrated along the subduction zones that define the Pacific Ring of Fire and parts of the Mediterranean/Asia Minor region. This highlights the regions of the world that have historically been capable of producing the most powerful volcanic events.

In [ ]:
def add_spatial_clusters(df, n_clusters=5):
    """
    Applies K-Means clustering to spatial coordinates.
    Adds a 'spatial_cluster' column to the dataframe.
    """
    df = df.copy()
    # Select coordinates
    X = df[['latitude', 'longitude']]

    # Apply K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['spatial_cluster'] = kmeans.fit_predict(X)

    # Convert to string for categorical coloring in plots
    df['spatial_cluster'] = df['spatial_cluster'].astype(str)

    return df

def get_indicator_spatial_clusters(df):
    """
    Visualizes the spatial clusters on a map.
    """
    fig = px.scatter_geo(
        df,
        lat='latitude',
        lon='longitude',
        color='spatial_cluster',
        hover_name='name',
        title='Spatial Clustering of Volcanoes (K-Means)',
        projection='natural earth',
        color_discrete_sequence=px.colors.qualitative.Prism
    )
    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    return fig

In [ ]:
add_spatial_clusters(df_final)

,year,tsu,eq,name,location,country,latitude,longitude,elevation_(m),type,...,total_houses_destroyed,century,has_vei,has_deaths,has_damage,vei_label,latitude_norm,longitude_norm,elevation_(m)_norm,spatial_cluster
1,-4360.0,0.0,0.0,Macauley,Kermadec Is,New Zealand,-30.210,-178.475,238.0,Caldera,...,NaN,-4400.0,1,0,0,6: Colossal,0.254755,0.000000,0.125374,4
2,-4350.0,0.0,0.0,Kikai,Ryukyu Is,Japan,30.793,130.305,704.0,Caldera,...,NaN,-4400.0,1,0,0,7: Super-colossal,0.728690,0.868201,0.191765,3
3,-4050.0,0.0,0.0,Masaya,Nicaragua,Nicaragua,11.985,-86.165,594.0,Caldera,...,NaN,-4100.0,1,0,0,6: Colossal,0.582569,0.259549,0.176093,2
4,-4000.0,0.0,0.0,Witori,New Britain-SW Pac,Papua New Guinea,-5.576,150.516,724.0,Caldera,...,NaN,-4000.0,1,0,0,6: Colossal,0.446137,0.925028,0.194615,0
5,-3580.0,0.0,0.0,Taal,Luzon-Philippines,Philippines,14.002,120.993,311.0,Stratovolcano,...,NaN,-3600.0,1,0,0,6: Colossal,0.598240,0.842018,0.135774,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,2024.0,0.0,0.0,Ijen,Java,Indonesia,-8.058,114.242,2769.0,Stratovolcano,...,NaN,2000.0,0,1,0,Unknown/Unrecorded,0.426854,0.823036,0.485967,0
884,2024.0,0.0,0.0,Marapi,Sumatra,Indonesia,-0.380,100.474,2885.0,Complex volcano,...,521.0,2000.0,0,1,0,Unknown/Unrecorded,0.486505,0.784325,0.502493,0
885,2024.0,0.0,0.0,Kanlaon,Philippines-C,Philippines,10.412,123.132,2435.0,Stratovolcano,...,2680.0,2000.0,0,0,1,Unknown/Unrecorded,0.570349,0.848033,0.438382,0
886,2024.0,5923.0,10744.0,Sheveluch,Kamchatka,Russia,56.653,161.360,3283.0,Stratovolcano,...,NaN,2000.0,1,0,0,4: Cataclysmic,0.929597,0.955519,0.559196,3


In [ ]:
get_indicator_spatial_clusters(df_final)

### **Indicator 3.B: Spatial Clustering (K-Means)**
Using unsupervised learning (K-Means), volcanoes have been grouped into geographical clusters based solely on their coordinates.  

Volcanoes are not randomly distributed. They are highly concentrated in distinct clusters, primarily along plate boundaries. The visualization clearly maps out the major global volcanic zones, such as the Pacific Ring of Fire (Clusters 0, 2, 3, 4), with sparser distributions in the interiors of continents.

## IV. VEI Distribution

This chart contains only the "global" part. In the dash we modified this function so it become interactive with a slider.

In [ ]:
def get_indicator_VEI_distribution(df_final):
    # Calculate counts, ensuring all VEI classes (0-8) are present and ordered.
    counts = df_final['vei_label'].value_counts() \
        .reindex(SORTED_VEI_LABELS, fill_value=0).reset_index()

    # Rename columns and filter out classes with zero eruptions.
    counts.columns = ['VEI Class', 'Count']
    counts = counts[counts['Count'] > 0]

    fig = px.pie(
        counts,
        values='Count',
        names='VEI Class',
        title="VEI Distribution: Total (All Recorded History)",

        category_orders={"VEI Class": SORTED_VEI_LABELS},

        color='VEI Class',
        color_discrete_sequence=px.colors.sequential.Inferno
    )

    fig.update_traces(sort=False)
    return fig

In [ ]:
get_indicator_VEI_distribution(df_final)

### **Indicator 4: VEI Distribution**
This pie chart illustrates the distribution of Volcanic Explosivity Index (VEI) levels for all recorded eruptions that occurred during the 1900s Century (1900-1999).  

The relative frequency (percentage) of different eruption intensities based on the VEI scale (0 to 6) in History.  

The vast majority of recorded eruptions are Explosive and Catastrophic eruptions.

## V. Temporal

In [ ]:
def get_indicator_temporal1(df):
    """
    Generates a stacked bar chart showing the number of volcanic eruptions per century
    (post-1200 AD), segmented by VEI class.

    This indicator visualizes the absolute count of recorded eruptions over time,
    providing evidence for the apparent increase in volcanic activity before
    considering observation bias.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing volcanic eruption data.
            It should include 'year', 'century', and 'vei_label' columns.

    Returns:
        plotly.graph_objects.Figure: A Plotly stacked bar chart displaying eruptions per century.
    """
    # Only 1200s onwards to focus on more reliably recorded history
    df_ctx = df[df['year'] >= 1200]
    # Group by century and VEI label to count eruptions for each category
    evolution = df_ctx.groupby(['century', 'vei_label']).size().reset_index(name='count')

    return px.bar(
        evolution, x='century', y='count', color='vei_label',
        title='Eruptions Per Century (Post-1200 AD)', barmode='stack',
        category_orders={"vei_label": SORTED_VEI_LABELS}
    )

In [ ]:
get_indicator_temporal1(df_final)

### **Temporal Indicator 5.A: Eruption Frequency**
This chart shows the absolute count of recorded eruptions per century, stacked by VEI class (post-1200 AD).  

The main takeaway is that the total number of recorded eruptions rises dramatically starting in the 1700s and accelerating into the 2000s. This provides the raw evidence for the **apparent increase in volcanic activity** over time. Without further context (Indicator 5.B), one might incorrectly conclude that activity is increasing.


In [ ]:
def get_indicator_accuracy(df):
    """
    Generates a line chart showing the evolution of data completeness over centuries.

    This indicator visualizes the 'observation bias' by tracking the percentage of
    eruptions with recorded VEI, Deaths, and Damage data, proving that the apparent
    increase in volcanic activity is correlated with improved data recording.

    Args:
        df (pd.DataFrame): The preprocessed DataFrame containing volcanic eruption data.
            It should include 'year', 'century', 'has_vei', 'has_deaths', and 'has_damage' columns.

    Returns:
        plotly.graph_objects.Figure: A Plotly line chart displaying data completeness trends.
    """
    # Filter for relevant history (post-1200 AD to match other graph)
    df_recent = df[df['year'] >= 1200]
    # Group by century and calculate the mean of 'has_vei', 'has_deaths', 'has_damage' to get completeness percentage
    accuracy = df_recent.groupby('century')[['has_vei', 'has_deaths', 'has_damage']].mean().reset_index()

    fig = go.Figure()
    # Add a trace for each data completeness metric (VEI, Deaths, Damage)
    for col, name in [('has_vei', 'VEI'), ('has_deaths', 'Deaths'), ('has_damage', 'Damage ($)')]:
        fig.add_trace(go.Scatter(x=accuracy['century'], y=accuracy[col], name=name, mode='lines+markers'))

    # Update layout for title and y-axis format
    fig.update_layout(title='Evolution of Data Completeness (Observation Bias Proof)', yaxis_tickformat='.0%')
    return fig

In [ ]:
get_indicator_accuracy(df_final)

### **Temporal Indicator 5.B: Accuracy Trend (Observation Bias Proof)**
This line chart shows the percentage of eruptions that have recorded data for VEI, Deaths, and Damage over the centuries.  

The completeness ratio for all metrics, particularly VEI, rises sharply from the 1700s, reaching nearly **100% in the modern era**. This trend directly mirrors the rise in reported eruption frequency (Indicator 5.A).  

This is the **proof of observation bias**. The increase in reported eruptions is directly correlated with the *increase in data completeness* (better reporting), confirming that **recorded activity has increased, but not necessarily actual activity.**  
* **Critical Note on Recent Data (Post-1990s):** The visual decline in VEI events in the 2000s is primarily due to a shift in modern geological monitoring. Many **small submarine or non-explosive events** are now recorded but are **not assigned a traditional VEI**. This technical change artificially lowers the recorded explosive VEI counts in the most recent periods, while keeping the total event count high.

## VI. Forecasting

In [ ]:
def get_indicator_forecasting(df):
    """
    Generates a forecasting chart using Linear Regression to predict
    the trend of *recorded* eruptions for the next 3 decades.

    Uses data from 1700 onwards to establish a stable modern trend.
    """
    # Prepare Data: Aggregate by Decade (Modern Era only)
    df_ts = df.copy()
    # Filter for post-1900 to focus on the industrial/technological age trend
    df_ts = df_ts[df_ts['year'] >= 1700]
    df_ts['decade'] = (df_ts['year'] // 10) * 10

    # Count eruptions per decade
    counts = df_ts.groupby('decade').size().reset_index(name='count')

    # Modeling (Linear Regression)
    # X needs to be 2D array [[1700], [1710]...]
    X = counts['decade'].values.reshape(-1, 1)
    y = counts['count'].values

    model = LinearRegression()
    model.fit(X, y)

    # Forecast: Predict next 3 decades (e.g., 2030, 2040, 2050)
    last_decade = counts['decade'].max()
    future_decades = np.array([last_decade + 10, last_decade + 20, last_decade + 30]).reshape(-1, 1)
    future_counts = model.predict(future_decades)

    # Visualization
    fig = go.Figure()

    # Trace 1: Historical Data (Solid Blue Line)
    fig.add_trace(go.Scatter(
        x=counts['decade'], y=counts['count'],
        mode='lines+markers', name='Historical Records',
        line=dict(color='royalblue', width=3)
    ))

    # Prepare data to connect the last historical point to the forecast
    forecast_x = np.concatenate(([X[-1][0]], future_decades.flatten()))
    forecast_y = np.concatenate(([y[-1]], future_counts))

    # Trace 2: Forecast Data (Dashed Red Line)
    fig.add_trace(go.Scatter(
        x=forecast_x, y=forecast_y,
        mode='lines+markers', name='Trend Forecast',
        line=dict(color='crimson', dash='dash', width=3)
    ))

    fig.update_layout(
        title="Forecasting: Trend of Recorded Eruptions (Linear Regression)",
        xaxis_title="Decade",
        yaxis_title="Number of Recorded Events",
        hovermode="x unified"
    )

    return fig

In [ ]:
get_indicator_forecasting(df_final)

### **Indicator 6 (Forecasting): Linear Regression on Time Series.**
We applied a Linear Regression model to the modern era (1900-2024) to forecast the trend of recorded eruptions.   

The model predicts a **continued increase** in the number of recorded events.  

As monitoring technology improves (satellites, AI), we expect to detect more small-scale eruptions that would have previously gone unnoticed.

# 5. DASHBOARD LAUNCHER


In [ ]:
def launch_dashboard(df_final):
    """
    Launches a Plotly Dash web application for visualizing volcanic activity and data completeness.

    The dashboard includes several interactive components:
    1. Global Ranking of Volcanic Activity (bar chart)
    2. Global Seismic Potential (scatter geo map)
    3. Interactive VEI Distribution (pie chart with a century slider)
    4. Eruption Frequency (stacked bar chart)
    5. Accuracy Trend (line chart for observation bias proof)

    Args:
        df_final (pd.DataFrame): The final preprocessed DataFrame containing volcanic eruption data.
    """
    app = Dash(__name__)

    # Slider Setup for interactive VEI distribution
    slider_centuries = sorted(df_final['century'].unique())

    # Custom Logic: Define special values for 'Pre-1700' and 'Total' on the slider
    modern_centuries = [1700, 1800, 1900, 2000]
    PRE_1700_VAL = 1600
    TOTAL_VAL = 2100

    # Combine historical, modern, and total values for slider steps
    slider_steps = [PRE_1700_VAL] + modern_centuries + [TOTAL_VAL]

    # Define marks for the slider, including custom labels for special values
    slider_marks = {
        PRE_1700_VAL: {'label': 'Pre-1700s', 'style': {'fontWeight': 'bold'}},
        TOTAL_VAL: {'label': 'Total', 'style': {'fontWeight': 'bold', 'color': 'red'}}
    }
    # Add marks for each modern century
    for c in modern_centuries:
        slider_marks[c] = str(c) + 's'

    # VERTICAL LAYOUT (100% Width for everything)
    app.layout = html.Div([
        html.H1("Volcanic Analysis: Activity vs. Observation Bias", style={'textAlign': 'center'}),
        html.H3(f"Team Leader: {TEAM_LEADER}", style={'textAlign': 'center'}),

        # 1. Frequent Pattern Mining (Sankey)
        html.H2("Frequent Pattern Mining (Rules Sankey)", style={'textAlign': 'center'}),
        html.Div([
            dcc.Graph(figure=get_indicator_pattern_mining(df_final))
        ], style={'width': '100%', 'padding': '10px'}),
        dcc.Markdown("""
        ### **Indicator 1: Association Rules Flow.**
        This Sankey diagram visualizes the probability flow of the Apriori algorithm.
        It maps the flow from specific VEI intensities (Antecedents on the left) to the Volcano Type (Consequent on the right)
        Width of the link = Confidence of the rule.

        The diagram reveals a massive convergence: VEI 2 (Explosive), VEI 3 (Catastrophic), VEI 4 (Cataclysmic), and even VEI 5 (Paroxysmic) all flow strongly into Stratovolcanoes.

        This visually proves that Stratovolcanoes are the primary geological drivers of explosive volcanic activity recorded in history. The width of the bands suggests a high confidence level for these rules, confirming that if a high-VEI event occurs, it is almost certainly a Stratovolcano
        """, style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),

        html.Hr(),

        # 2 Grouping
        html.H2("Grouping", style={'textAlign': 'center'}),
        html.Div([
            dcc.Graph(figure=get_indicator_grouping(df_final), style={'height': '700px'})
        ], style={'width': '100%', 'padding': '20px 0'}),
        dcc.Markdown("""
        ### **Indicator 2: Grouping - Global Activity Ranking**
        This chart identifies the countries with the highest number of volcanic events.
        It establishes the major areas of focus for volcanic activity globally and provides the necessary **geographical context** for understanding where volcanic activity is concentrated, helping to interpret the spatial map (Indicator 2).
        """, style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),

        html.Hr(),

        # 3.A Spatial (Map)
        html.H2("Spatial Analysis", style={'textAlign': 'center'}),
        html.H3("Global Seismic Potential (Max VEI)", style={'textAlign': 'center'}),
        html.Div([
            dcc.Graph(figure=get_indicator_spatial(df_final), style={'height': '700px'})
        ], style={'width': '100%', 'padding': '20px 0'}),
        dcc.Markdown(
            """
            ### **Indicator 3.A: Spatial - Global Explosive Potential**
            The map plots every recorded volcano, with the marker size and color representing the maximum recorded **Volcanic Explosivity Index (VEI)**.


            The largest (highest VEI) eruptions, indicated by the largest and darkest red markers (VEI 6 and 7), are highly concentrated along the subduction zones that define the Pacific Ring of Fire and parts of the Mediterranean/Asia Minor region. This highlights the regions of the world that have historically been capable of producing the most powerful volcanic events.
            """,
            style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),

        # 3.B Spatial Clustering (Map)
        html.H3("Spatial Clustering", style={'textAlign': 'center'}),
        html.Div([dcc.Graph(figure=get_indicator_spatial_clusters(df_final))], style={'width': '100%', 'padding': '20px 0'}),

        dcc.Markdown("""
            ### **Indicator 3.B: Spatial Clustering (K-Means)**
            Using unsupervised learning (K-Means), volcanoes have been grouped into geographical clusters based solely on their coordinates.

            Volcanoes are not randomly distributed. They are highly concentrated in distinct clusters, primarily along plate boundaries. The visualization clearly maps out the major global volcanic zones, such as the Pacific Ring of Fire (Clusters 0, 2, 3, 4), with sparser distributions in the interiors of continents.
            """,
            style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),

        # 4. Interactive Transformation (Pie + Slider)
        html.H2("Interactive Explosivity Distribution (Sorted VEI)", style={'textAlign': 'center'}),
        html.Div([
            dcc.Graph(id='vei-pie-output', style={'height': '600px'}),
            html.Div([
                dcc.Slider(
                    id='century-slider',
                    min=min(slider_steps), max=max(slider_steps),
                    step=100, value=1900, marks=slider_marks,
                    tooltip={"placement": "bottom", "always_visible": False}
                )
            ], style={'width': '80%', 'margin': '0 auto', 'paddingBottom': '40px'})
        ], style={'width': '100%'}),
        dcc.Markdown(
            """
            ### **Indicator 4: VEI Distribution**
            This pie chart illustrates the distribution of Volcanic Explosivity Index (VEI) levels for all recorded eruptions that occurred during the 1900s Century (1900-1999).

            The relative frequency (percentage) of different eruption intensities based on the VEI scale (0 to 6) in History.

            Use the slider below to dynamically filter the Pie Chart by time period, including a **'Pre-1700s'** historical group and a **'Total'** overview. By moving the slider from 'Pre-1700s' towards the '2000s', observe how the distribution shifts, typically showing a decrease in low-VEI events and a relative increase in mid-range events. This is the **central hypothesis visual**. The increase in reported lower-magnitude events is expected if monitoring improves (Observation Bias).
            """,
            style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),


        html.Hr(),

        # 5.A Temporal (Stacked Bar + Accuracy)

        html.H2("Temporal Analysis", style={'textAlign': 'center'}),
        html.Div([
            dcc.Graph(figure=get_indicator_temporal1(df_final))
        ], style={'width': '100%', 'padding': '20px 0'}),
        dcc.Markdown(
            """
            ### **Temporal Indicator 5.A: Eruption Frequency**
            This chart shows the absolute count of recorded eruptions per century, stacked by VEI class (post-1200 AD).The main takeaway is that the total number of recorded eruptions rises dramatically starting in the 1700s and accelerating into the 2000s. This provides the raw evidence for the **apparent increase in volcanic activity** over time. Without further context (Indicator 5.B), one might incorrectly conclude that activity is increasing.
            """,
            style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),

        # 5.B Accuracy Trend (Line Chart)
        html.Div([
            dcc.Graph(figure=get_indicator_accuracy(df_final))
        ], style={'width': '100%', 'padding': '20px 0'}),
        dcc.Markdown(
            """
            ### **Temporal Indicator 5.B: Accuracy Trend (Observation Bias Proof)**
            This line chart shows the percentage of eruptions that have recorded data for VEI, Deaths, and Damage over the centuries.

            The completeness ratio for all metrics, particularly VEI, rises sharply from the 1700s, reaching nearly **100% in the modern era**. This trend directly mirrors the rise in reported eruption frequency (Indicator 5.A).

            This is the **proof of observation bias**. The increase in reported eruptions is directly correlated with the *increase in data completeness* (better reporting), confirming that **recorded activity has increased, but not necessarily actual activity.**
            * **Critical Note on Recent Data (Post-1990s):** The visual decline in VEI events in the 2000s is primarily due to a shift in modern geological monitoring. Many **small submarine or non-explosive events** are now recorded but are **not assigned a traditional VEI**. This technical change artificially lowers the recorded explosive VEI counts in the most recent periods, while keeping the total event count high.
            """,
            style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),

        # 5.C Future Trend Forecasting

        html.H3("Future Trend Forecasting", style={'textAlign': 'center'}),
        html.Div([
            dcc.Graph(figure=get_indicator_forecasting(df_final))
        ], style={'width': '100%', 'padding': '10px'}),
        dcc.Markdown("""
        ### **Indicator 5.C (Forecasting): Linear Regression on Time Series.**
        We applied a Linear Regression model to the modern era (1900-2024) to forecast the trend of recorded eruptions.

        The model predicts a **continued increase** in the number of recorded events.

        As monitoring technology improves (satellites, AI), we expect to detect more small-scale eruptions that would have previously gone unnoticed.
                """,
        style={'maxWidth': '900px', 'margin': '0 auto', 'marginBottom': '40px'}
        ),


    ], style={'maxWidth': '1200px', 'margin': '0 auto'})

    @app.callback(Output('vei-pie-output', 'figure'), Input('century-slider', 'value'))
    def update_pie(slider_value):
        # Filtering Logic based on slider value
        if slider_value == PRE_1700_VAL:
            subset = df_final[df_final['century'] < 1700]
            title_text = "VEI Distribution: Pre-1700s (Historical)"
        elif slider_value == TOTAL_VAL:
            subset = df_final
            title_text = "VEI Distribution: Total (All Recorded History)"
        else:
            subset = df_final[df_final['century'] == slider_value]
            title_text = f"VEI Distribution: {int(slider_value)}s Century"

        # Count VEI labels and reindex to maintain consistent order
        counts = subset['vei_label'].value_counts().reindex(SORTED_VEI_LABELS, fill_value=0).reset_index()
        counts.columns = ['VEI Class', 'Count']
        # Filter out VEI classes with zero counts for cleaner visualization
        counts = counts[counts['Count'] > 0]

        # Create a pie chart with Plotly Express
        fig = px.pie(
            counts, values='Count', names='VEI Class',
            title=title_text,
            category_orders={"VEI Class": SORTED_VEI_LABELS},
            color='VEI Class',
            color_discrete_sequence=px.colors.sequential.Inferno
        )
        fig.update_traces(sort=False)
        return fig

    print("Dashboard created. Running...")

    app.run(host='127.0.0.1', debug=True, jupyter_mode='inline')


# 6. MAIN EXECUTION

In [ ]:
if __name__ == "__main__":
    # Load the raw dataset
    df_raw = load_data(DATASET_PATH)
    if df_raw is not None:
        # Perform initial data exploration and display summary statistics
        data_collection_exploration(df_raw)

        # Build and apply the data cleaning and transformation pipeline
        pipeline = build_pipeline()
        df_final = pipeline.fit_transform(df_raw)

        print("Pipeline Complete.")
        # Launch the interactive Dash dashboard with the processed data
        launch_dashboard(df_final)


[INFO] Dataset dimensions: 888 rows, 35 columns

[INFO] Data types per column:
Search Parameters                      object
Year                                  float64
Mo                                    float64
Dy                                    float64
Tsu                                   float64
Eq                                    float64
Name                                   object
Location                               object
Country                                object
Latitude                              float64
Longitude                             float64
Elevation (m)                         float64
Type                                   object
VEI                                   float64
Agent                                  object
Deaths                                float64
Death Description                     float64
Missing                               float64
Missing Description                   float64
Injuries                              float64


INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: on
